In [1]:
from data_lib.data_lib import *

In [2]:
some_letters = read_data('AOMBPZKCEF.csv')

In [3]:
letters_map = {'A':1,'O':2,'M':3,'B':4,'P':5,'Z':6,'K':7,'C':8,'E':13,'F':14}

In [4]:
gests = read_data('data2.csv')

In [5]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Reshape
from keras.layers import LSTM

Using Theano backend.


In [6]:
n_units=100
n_features=3
n_epoch=10
model = Sequential()  # Creates a model
model.add(LSTM(n_units, input_dim=n_features)) 
model.add(Dense(16, activation='sigmoid'))  # Kind of output layer
model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

In [7]:
model.load_weights('gests_lstm_weights')

In [8]:
def letters_predicted_sequences(model, letters, win_size=15, proba_threshold=0.5):
    X = []
    letters_array, y = array_from_dict(letters, ndots=-1, gyro=False)
    for letter in letters_array:
        seq = []
        k = 0
        while len(letter[0])>k+win_size:
            p = model.predict_proba(np.array([letter[0][k:k+win_size]]), verbose=0)[0]
            p = p / np.sum(p)
            p = np.where(p > 0.5)[0]
            if p:
                seq.append(p)
            k += 1
        X.append(np.array([seq]))
    return X, y
        

In [9]:
some_letters = add_noise(some_letters, 0.5, 100)

In [10]:
r,t = letters_predicted_sequences(model, some_letters)

In [11]:
bufx, bufy = [],[]
for (x,y) in zip(r, t):
    if x[0].shape[0]!=0:
        bufx.append(x)
        bufy.append(y)
r=bufx
t=bufy

In [12]:
map_eee = {1:1,2:2,3:3,4:4,5:5,6:6,7:7,8:8,13:9,14:10}

In [13]:
t1 = np.array([map_eee[x] for x in t])

In [14]:
b = np.zeros((len(t), 10))
b[np.arange(len(t)), t1-1] = 1
t1=b

In [15]:
n_units=50
n_features=1
n_epoch=7
model1 = Sequential()  # Creates a model
model1.add(LSTM(n_units, input_dim=n_features)) 
model1.add(Dense(10, activation='sigmoid'))  # Kind of output layer
model1.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

In [16]:
print(model1.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lstm_2 (LSTM)                    (None, 50)            10400       lstm_input_2[0][0]               
____________________________________________________________________________________________________
dense_2 (Dense)                  (None, 10)            510         lstm_2[0][0]                     
Total params: 10910
____________________________________________________________________________________________________
None


In [17]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(r, t1, test_size=0.2)

for seq, label in zip(X_train, y_train):
    label = label.reshape((1, -1))
    model1.fit(seq, label, nb_epoch=n_epoch, batch_size=1, verbose=0)

In [18]:
y_pred = []
y_true = []
scores = []
for seq, label in zip(X_test, y_test):
    label = label.reshape((1, -1))
    scores.append(model1.evaluate(seq, label, verbose=0)[1])
    y_pred.append(model1.predict_classes(seq, verbose=0)[0])
    y_true.append(np.where(label == 1)[1][0])

In [19]:
np.mean(scores)

0.70909090909090911